## Offline AI LLM


### Firstly we are going to get text to speech working

In [50]:
# for tts we are going to install pyttsx3:
# !pip install pyttsx3

#### ok here we test typing in a name for the engine to speak back

In [44]:
import pyttsx3  
import time
# initialize Text-to-speech engine  
engine = pyttsx3.init()  
# convert this text to speech  

text = "hey, whats your name?"  
engine.say(text)    
engine.runAndWait()
# play the speech
name=input()
text1 = name + ", how are you doing?"  
engine.say(text1)  
# play the speech  
engine.runAndWait()


Batman


#### what voices do we have at our disposal

In [49]:
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')
for voice in voices:
    print(voice, voice.id)


<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-GB_HAZEL_11.0
          name=Microsoft Hazel Desktop - English (Great Britain)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-GB_HAZEL_11.0
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
          name=Microsoft Zira Desktop - English (United States)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0


### OK now we need a voice recognition tool, we go with vosk because its offline. 

In [ ]:
# the below worked for me, url is here: https://alphacephei.com/vosk/install
# !pip install pyaudio
# !pip install vosk

In [51]:
import vosk
import pyaudio
import json

model = vosk.Model(lang="en-us")

In [52]:
rec = vosk.KaldiRecognizer(model, 16000)

In [53]:
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=1024,
                input_device_index=0)

In [54]:
output_file_path = "recognized_text.txt"

In [58]:
# Open a text file in write mode using a 'with' block

with open(output_file_path, "w") as output_file:
    print("Listening for speech. Say 'Hazel' to stop.")
    # Start streaming and recognize speech
    while True:
        data = stream.read(1024)#read in chunks of 4096 bytes
        if rec.AcceptWaveform(data):#accept waveform of input voice
            # Parse the JSON result and get the recognized text
            result = json.loads(rec.Result())
            recognized_text = result['text']
            
            # Write recognized text to the file
            output_file.write(recognized_text + "\n")
            print(recognized_text)
            
            # Check for the termination keyword
            if "hazel" in recognized_text.lower():
                print("Termination keyword detected. Stopping...")
                break

Listening for speech. Say 'Hazel' to stop.
i wonder if this speech recognition works hazel
Termination keyword detected. Stopping...


In [59]:
recognized_text.replace('hazel', '')

'i wonder if this speech recognition works '

In [60]:
# Stop and close the stream
stream.stop_stream()
stream.close()

# Terminate the PyAudio object
p.terminate()

In [67]:
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

for i in range(0, numdevices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Microphone Array (Realtek High 


#### LLM

In [16]:
# !pip install ollama

  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.11.0
    Uninstalling typing-extensions-4.11.0:
      Successfully uninstalled typing-extensions-4.11.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic-core 2.20.1
    Uninstalling pydantic-core-2.20.1:
      Successfully uninstalled pydantic-core-2.20.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2


### Now pull it all together

In [ ]:
import ollama
import pyttsx3  
import time

In [68]:
# lets get some interaction with the tinyllama package of ollama:

get_answer = ollama.generate(model='tinyllama',prompt='explain what quantum computing is in 2 sentences')
text = get_answer['response'] 
print(text)

Quantum computing is an emerging field that involves using quantum mechanics to perform complex computations, which can be much faster and more efficient than traditional classical computing methods. In 2 sentences:
- It's a cutting-edge technology that uses the laws of quantum mechanics to solve problems that would take days or months on classical computers. - Quantum computing is transforming fields such as physics, chemistry, materials science, finance, and healthcare by opening up new possibilities for solving complex problems in record times.


In [77]:
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=1024,
                input_device_index=0)

#### drum roll.............

In [ ]:
engine = pyttsx3.init()  
with open(output_file_path, "w") as output_file:
    print("Listening for speech. Say 'Hazel' to stop.")
    while True:
        data = stream.read(1024)
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            recognized_text = result['text']
                
            output_file.write(recognized_text + "\n")
            print(recognized_text)
        
            if "hazel" in recognized_text.lower():
                ques = recognized_text.replace('hazel', '')
                get_answer = ollama.generate(model='tinyllama',prompt=ques)
                text = get_answer['response'] 
                print(text)
                engine.say(text)    
                engine.runAndWait()
                engine.stop()
                break



Listening for speech. Say 'Hazel' to stop.

can you explain to me what quantum computing is hazel
Quantum computing is a type of computing that involves manipulating and processing information using quantum mechanical properties, which involve the behaviour of atoms and molecules in ways that are inaccessible to classical computers. In contrast to classical computers, where each bit (binary digit) can have only two possible states (0 or 1), quantum bits (qubits) can exist in a superposition of several possible states, with each state carrying information in its own unique manner.

The process of computing using quantum algorithms involves the application of specialized hardware and software to perform calculations at a much faster rate than classical computers. Quantum algorithms use quantum mechanics, which is based on the principles of wave function collapse and superposition, to solve complex problems that are not easily solved by classical computers. These algorithms can be used in

In [ ]:
# being responsible
stream.stop_stream()
stream.close()
p.terminate()